В этом ноутбуке я обработал данные вакансий для загрузки в базу данных и дальнейшего моделирования.
Шаг обработки следующие:
- ‌id — проверка и удаление дубликатов

- name — привести к нижнему регистру, убрать стопслова, убрать грейды

- city — убрать записи с лишними городами 

- salary — удалить слишком маленькие значения (=валюта не рубли), заполнить пустые значения from/to значениями to/from, добавление медианного значения (mid_salary)

- employer — удалить строки с ИП и ФИО, НИЦ/НИИ/НТЦ привести к значению "институты", убрать из названий ГК/ООО/АО и т.д.

- work_format — всё, что содержит "гибрид", заменить на гибрид, где remote — удалённо, on_site — в офисе, в остальных случаях, если есть field work, то удалить записи

- experience — преобразовать, добавить фичу: нет опыта — intern/junior, 1-3 — jun+/middle, 3-6 — middle+/senior

- ‌description — извлечь раздел требования, привести к нижнему регистру, удалить знаки препинания, добавить в пустые ячейки key_skills

- key_skills — сгруппировать подсчёт key_skills по ролям

In [1]:
# Импортируем библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# Загружаем сырые данные и выводим основную информацию
cities = pd.read_excel(r'Z:\Repos\mlops2025\hr_analytics\data\raw\cities.xlsx')
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2191 non-null   int64  
 1   name         2191 non-null   object 
 2   group        2191 non-null   object 
 3   city         2191 non-null   object 
 4   salary_from  1690 non-null   float64
 5   salary_to    1464 non-null   float64
 6   employer     2191 non-null   object 
 7   work_format  2191 non-null   object 
 8   experience   2191 non-null   object 
 9   key_skills   2191 non-null   object 
 10  description  2191 non-null   object 
dtypes: float64(2), int64(1), object(8)
memory usage: 188.4+ KB


In [98]:
cities.shape

(2191, 11)

In [99]:
# Посмотрим на первые 5 строк
cities.head()

,id,name,group,city,salary_from,salary_to,employer,work_format,experience,key_skills,description
0,126298182,Системный/бизнес аналитик (стажер),Системный аналитик,Казань,NaN,23200.0,ICL Services,['REMOTE'],Нет опыта,"['Аналитическое мышление', 'Системный анализ',...",Мы – российская продуктово-сервисная ИТ-компан...
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,NaN,ТАТТЕЛЕКОМ,['ON_SITE'],От 1 года до 3 лет,[],ПАО «Таттелеком» — один из ключевых операторов...
2,126319001,QA-инженер (senior),Тестировщик,Казань,150000.0,170000.0,ГК Innostage,['REMOTE'],От 3 до 6 лет,"['REST', 'SOAP', 'SQL', 'Git', 'Ручное тестиро...","Компания Innostage (Инностейдж) - IT компания,..."
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,Кинофлекс,['REMOTE'],От 1 года до 3 лет,"['Figma', 'UI', 'UX']",Компания FLEX — это инновационный онлайн-кинот...
4,126243205,Junior Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,Первый Бит,[],От 1 года до 3 лет,"['Python', 'Docker', 'SQL', 'FastAPI']",Обязанности: Поддержка и развитие чат-бота в ...


# Обработка id

In [127]:
# Проверим, нет ли повторяющихся вакансий
# Если они есть, удалим их
df_duplicates = cities[cities.duplicated(subset=['id'], keep=False)].sort_values(by='id')
df_duplicates.shape  # 50 записей - дубликаты

(50, 11)

In [128]:
# Удалим дубликаты
vacancies = cities.copy()  # работаем с копией данных
vacancies.drop_duplicates(subset='id', keep='first', inplace=True)
vacancies.shape

(2166, 11)

# Обработка городов

In [129]:
# Проверим, нет ли лишних городов
vacancies['city'].unique()

array(['Казань', 'Москва', 'Санкт-Петербург', 'Уфа', 'Екатеринбург',
       'Краснодар', 'Новосибирск', 'Пермь', 'Тюмень', 'Ноябрьск',
       'Челябинск', 'Первоуральск', 'Ревда (Свердловская область)',
       'Академгородок (Новосибирская область)', 'Бердск', 'Сочи',
       'Ломоносов', 'Белоостров', 'Колпино', 'Обнинск', 'Зеленоград',
       'Жуковский  (Московская область)', 'Томск', 'Балаково', 'Оренбург',
       'Дмитров (Московская область)', 'Лобня (Московская область)',
       'Самара', 'Курск', 'Воронеж', 'Тверь'], dtype=object)

In [130]:
# Удалим записи с лишними городами
city_list = ['Москва', 'Санкт-Петербург', 'Казань', 'Новосибирск', 'Екатеринбург']
vacancies = vacancies[vacancies['city'].isin(city_list)].reset_index(drop=True)

# Проверка городов
vacancies['city'].unique()


array(['Казань', 'Москва', 'Санкт-Петербург', 'Екатеринбург',
       'Новосибирск'], dtype=object)

# Обработка зарплат

In [131]:
# Заполним пустые значения from и to значениями to и from

# Заполняем нижнюю границу верхней, если поле пустое
vacancies['salary_from'] = vacancies['salary_from'].fillna(vacancies['salary_to'])

# То же самое с верхней границей
vacancies['salary_to'] = vacancies['salary_to'].fillna(vacancies['salary_from'])

# Заполняем медианой новый столбец: средним списка из двух
vacancies['mid_salary'] = np.median([vacancies['salary_from'], vacancies['salary_to']], axis=0)

# Перегруппируем столбцы для лучшего отображения
cols = vacancies.columns.to_list()
vac = vacancies[['id',
           'name',
           'group',
           'city',
           'salary_from',
           'salary_to',
           'mid_salary',
           'employer',
           'work_format',
           'experience',
           'key_skills',
           'description',
           ]]

vac.head()

,id,name,group,city,salary_from,salary_to,mid_salary,employer,work_format,experience,key_skills,description
0,126298182,Системный/бизнес аналитик (стажер),Системный аналитик,Казань,23200.0,23200.0,23200.0,ICL Services,['REMOTE'],Нет опыта,"['Аналитическое мышление', 'Системный анализ',...",Мы – российская продуктово-сервисная ИТ-компан...
1,126267340,Разработчик,"Программист, разработчик",Казань,175500.0,175500.0,175500.0,ТАТТЕЛЕКОМ,['ON_SITE'],От 1 года до 3 лет,[],ПАО «Таттелеком» — один из ключевых операторов...
2,126319001,QA-инженер (senior),Тестировщик,Казань,150000.0,170000.0,160000.0,ГК Innostage,['REMOTE'],От 3 до 6 лет,"['REST', 'SOAP', 'SQL', 'Git', 'Ручное тестиро...","Компания Innostage (Инностейдж) - IT компания,..."
3,126271542,Дизайнер интерфейсов,"Дизайнер, художник",Казань,140000.0,150000.0,145000.0,Кинофлекс,['REMOTE'],От 1 года до 3 лет,"['Figma', 'UI', 'UX']",Компания FLEX — это инновационный онлайн-кинот...
4,126243205,Junior Backend Engineer AI,"Программист, разработчик",Казань,60000.0,80000.0,70000.0,Первый Бит,[],От 1 года до 3 лет,"['Python', 'Docker', 'SQL', 'FastAPI']",Обязанности: Поддержка и развитие чат-бота в ...


In [132]:
vac.shape

(2129, 12)

In [133]:
# Удалим значения со слишком малыми значениями
vac = vac[vac['mid_salary'] > 10000]
vac.shape

(2077, 12)

# Обработка названий

In [87]:
pd.set_option('display.max_rows', None)

In [89]:
# Подбираем список стоп-слов
pd.set_option('display.max_rows', None)
freq_words_titles = pd.Series(' '.join(vacancies['name']).lower().split()).value_counts()
freq_words_titles

разработчик                                     284
1с                                              270
аналитик                                        243
системный                                       236
/                                               194
по                                              187
программист                                     176
инженер                                         176
администратор                                   156
developer                                       144
senior                                          138
специалист                                      131
ведущий                                         129
engineer                                        100
в                                                90
middle                                           82
qa                                               80
и                                                68
backend                                          56
инженер-прог

In [ ]:
def remove_parentheses(text):
    # Удаляет круглые скобки и всё, что внутри
    # Включает вложенные случаи, но удаляет только один уровень
    title = text.lower().strip()
    title = re.sub(r'\([^)]+\)', '', text)
    
    return 

vacancies['name'] = vacancies['name'].apply(remove_parentheses)
freq_words_titles = pd.Series(' '.join(vacancies['name']).lower().split()).value_counts()
freq_words_titles

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [12]:
vacancies[vacancies['name'].str.contains('по')].head()

,id,name,group,city,salary_from,salary_to,employer,work_format,experience,key_skills,description,mid_salary
8,126161071,Специалист по тестированию (Тестировщик),Тестировщик,Казань,40000.0,40000.0,Системы документооборота,['ON_SITE'],Нет опыта,"['Ручное тестирование', 'Тестирование ПО']","У тебя высшее образование и средний балл от 4,...",40000.0
32,125471160,QA инженер по автоматизации (Python),Тестировщик,Казань,150000.0,200000.0,НДК,['REMOTE'],От 1 года до 3 лет,"['Pytest', 'Python']",Обязанности:1. Разработка и сопровождение авто...,175000.0
42,125882889,Аналитик / Специалист технической поддержки,Аналитик,Казань,35000.0,40000.0,Hotmaps,['REMOTE'],Нет опыта,"['Аналитическое мышление', 'MS Excel', 'Англий...",Продуктовая компания Hotmaps разрабатывает сер...,37500.0
46,126102500,Инженер по качеству сети,Сетевой инженер,Казань,60000.0,60000.0,ТАТТЕЛЕКОМ,[],От 1 года до 3 лет,[],«ТАТТЕЛЕКОМ» ВХОДИТ В ТОП-15 КРУПНЕЙШИХ ТЕЛЕКО...,60000.0
53,124893002,Инженер по информационной безопасности,Специалист по информационной безопасности,Казань,70000.0,100000.0,Центр информационных технологий Республики Тат...,['ON_SITE'],От 3 до 6 лет,[],1. Участие во внедрении средств защиты информа...,85000.0


# Обработка работодателя

In [134]:
pd.set_option('display.max_rows', None)

In [166]:
# Посмотрим на количество уникальных работодателей
uni_employers = vac['employer'].value_counts()
uni_employers

employer
научные предприятия                                    117
первый бит                                              64
линсофтверпродакшн                                      50
иц ай-теко                                              47
госкорпорация                                           41
aston                                                   24
специальный технологический центр                       18
моринформсистема-агат                                   18
dcloud                                                  17
бизнес технологии                                       15
мтс                                                     15
funflow                                                 14
ростелеком                                              13
тензор                                                  12
skillstaff                                              11
evrone.ru                                               10
клируэй текнолоджис                            

In [148]:
# Обработаем столбец с работодателями

vac['employer'] = vac['employer'].str.lower()

# Унифицируем научные предприятия
pat_science = r"НПЦ|НИИ|НТЦ|НПО|научно-производственное объединение|НПК|Научно-Исследовательский Институт|ВНИИ|НПП|НИЦ|лаборатория|научно-производственное предприятие|информационный центр|нтпц".lower()
vac['employer'] = vac['employer'].where(~vac['employer'].str.contains(pat_science, case=False, na=False), "научные предприятия")

# Унифицируем государственные предприятия
pat_state = r"ФКУ|ФГАНУ|ФГУП|ФГБУ|Городской информационный центр|Республики|Филиал|гку|оцрв|ано|фбу|гбу".lower()
vac['employer'] = vac['employer'].where(~vac['employer'].str.contains(pat_state, case=False, na=False), "госкорпорация")

vac['employer'] = vac['employer']

In [ ]:
vac['employer'] = vac['employer'].str.replace(r"\([^)]*\)", "", regex=True)
vac['employer'] = vac['employer'].str.replace(r",.*$", "", regex=True)
vac['employer'] = vac['employer'].where(~vac['employer'].str.contains('газпром'), 'газпром')
vac['employer'] = vac['employer'].where(~vac['employer'].str.contains('сбер') & ~vac['employer'].str.contains('sber'), 'сбер')
vac['employer'] = vac['employer'].str.replace(r"/.*$", "", regex=True)
vac['employer'] = vac['employer'].str.replace(r"\|.*$", "", regex=True)


vac['employer'] = vac['employer'].str.strip()

In [174]:
name_list = [
    'мухин дмитрий сергеевич',
    'чепурин алексей владимирович',
    'дрынкин андрей александрович',
    'сизинцев роман александрович',
    'колпаков денис викторович',
    'лебедев роман игоревич',
    'токарева александра маратовна',
    'буров алексей константинович',
    'харламов денис сергеевич',
    'бабаян денис маркович',
    'шиховец александр викторович',
    'росляков никита александрович',
    'коваленко юрий николаевич',
    'мамедов эльдар юнисович',
    'бордаковский антон робертович',
]

In [175]:
vac = vac[~vac['employer'].isin(name_list)]

In [177]:
vac[vac['employer'] == 'инженерный центр информационно-аналитических систем']

,id,name,group,city,salary_from,salary_to,mid_salary,employer,work_format,experience,key_skills,description
1246,126126809,Системный администратор Linux,Системный администратор,Москва,150000.0,200000.0,175000.0,инженерный центр информационно-аналитических с...,['ON_SITE'],От 1 года до 3 лет,"['Администрирование сетевого оборудования', 'Н...","Уважаемые кандидаты, вакансия предполагает нах..."
1652,125644095,Разработчик Node.js,"Программист, разработчик",Москва,300000.0,300000.0,300000.0,инженерный центр информационно-аналитических с...,"['REMOTE', 'HYBRID']",От 3 до 6 лет,"['JavaScript', 'Node.js', 'TypeScript', 'Postg...",Сегодня перед компанией &quot;ИЦ ИАС&quot; пос...
